<a href="https://colab.research.google.com/github/ratmcu/wiki_ner/blob/master/segregation_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wget
import os
import wget
import logging
import torch
logger = logging.getLogger()
logger.setLevel(logging.INFO)
try:
    import colabimport
except:
    colabimporturl = 'https://github.com/ratmcu/colaboratory_import/raw/master/colabimport.py'
    filename = colabimporturl.split("/")[-1].split("?")[0]
    if os.path.isfile(filename):
        os.remove(filename)
    wget.download(colabimporturl)
    import colabimport
    
colabimport.get_notebook('https://github.com/ratmcu/wiki_ner/blob/master/dataset_results.ipynb?raw=true')
# colabimport.get_notebook('https://github.com/ratmcu/wiki_ner/blob/master/remote_bert_wikiner.ipynb?raw=true')
# from dataset_results import metric_calc, Net 

In [9]:
#experiment_code
import tarfile
!pip install wget
import os
import wget
wget.download('http://archive.org/download/wikiner_dataset_csv.tar/wikiner_dataset_txt.tar.gz')
tar = tarfile.open('wikiner_dataset_txt.tar.gz', mode='r')
tar.extractall('./dataset_txt')
tar.close()

In [0]:
#experiment_code
wget.download('https://archive.org/download/auto_manual_annots.tar/auto_manual_annots.tar.gz')
tar =  tarfile.open('./auto_manual_annots.tar.gz', mode='r')
tar.extractall('./auto_manual_annots')
tar.close()

In [0]:
#experiment_code
paths_annot = sorted([os.path.join(f[0], name) for f in os.walk('./dataset_txt') 
                if len(f[2])!=0 for name in f[2] if os.path.splitext(name)[-1] == '.txt' and name.split('_')[0]=='annot'],
               key=lambda path: int(path.split('_')[-1].split('.')[0]))
training_files = [path for path in paths_annot if 'training' in path.split('/')]
eval_files = [path for path in paths_annot if 'eval' in path.split('/')]
test_files = [path for path in paths_annot if 'test' in path.split('/')]

In [3]:
#experiment_code
from dataset_results import NerDataset, pad, data 
eval_dataset = NerDataset(eval_files[0])
_ = [eval_dataset.append(NerDataset(eval_file)) for eval_file in eval_files[1:]]

test_dataset = NerDataset(test_files[0])
_ = [test_dataset.append(NerDataset(test_file)) for test_file in test_files[1:]]

train_dataset = NerDataset(training_files[0])
_ = [train_dataset.append(NerDataset(train_file)) for train_file in training_files[1:]]

eval_iter = data.DataLoader(dataset=eval_dataset,
                              batch_size=1,
                              shuffle=True,
                              num_workers=16,
                              collate_fn=pad)

test_iter = data.DataLoader(dataset=test_dataset,
                              batch_size=32,
                              shuffle=True,
                              num_workers=16,
                              collate_fn=pad)

train_iter = data.DataLoader(dataset=train_dataset,
                              batch_size=1,
                              shuffle=True,
                              num_workers=16,
                              collate_fn=pad)

print(len(train_iter))
print(len(eval_iter))
print(len(test_iter))

importing Jupyter notebook from dataset_results.ipynb
man_test_iter iterator size: 320
aut_test_iter iterator size: 426
77703
12267
65


In [0]:
tags = ['BD', 'BP', 'PR', 'SP', 'CH', 'ED']
VOCAB_list = ['<PAD>', 'O',]
for tag in tags:
    VOCAB_list.append('I-'+tag)
    VOCAB_list.append('B-'+tag)
VOCAB = tuple(VOCAB_list)
tag2idx = {tag: idx for idx, tag in enumerate(VOCAB)}
idx2tag = {idx: tag for idx, tag in enumerate(VOCAB)}

class SegNerDataset(data.Dataset):
    def __init__(self, entity):
        self.entity = entity
        self.percentage = 1
        self.sents = []
        self.tags_li = []
        self.x = []
        self.y = []
        self.is_heads = []
        self.seqlen = []

    def __len__(self):
        return int(len(self.sents)*self.percentage)

    def __getitem__(self, idx):
        words, tags = self.sents[idx], self.tags_li[idx] 
        x, y = self.x[idx], self.y[idx]  
        is_heads, seqlen = self.is_heads[idx], self.seqlen[idx] 
        return words, x, is_heads, tags, y, seqlen
    
    def append(self, other):
        self.sents.extend(other.sents)
        self.tags_li.extend(other.tags_li)
        self.x.extend(other.x)
        self.y.extend(other.y)
        self.is_heads.extend(other.is_heads)
        self.seqlen.extend(other.seqlen)
    
    def add(self, entry):
        self.sents.extend(entry[0])
        self.x.extend(entry[1])
        self.is_heads.extend(entry[2])
        self.tags_li.extend(entry[3])
        self.y.extend(entry[4])
        self.seqlen.extend(entry[5])
    
    def setContribution(self, percentage):
        self.percentage = percentage

In [0]:
#experiment_code
# getting the indices relavant to a class.
# from dataset_results import train_dataset
from collections import defaultdict
tag_idx_dict = {'BD':[], 'BP':[], 'PR':[], 'SP':[], 'CH':[], 'ED':[]}
segregated_ds = defaultdict()

for key in tag_idx_dict.keys():
    segregated_ds[key] = SegNerDataset(key)

for i, entry in enumerate(train_iter):
    # print(entry)
    words, x, is_heads, tags, y, seqlens = entry
    tags = tags[0].split()
    for tag in tags:
        for key in tag_idx_dict.keys():
            if tag[2:] == key:
                tag_idx_dict[key].append(i)
                # segregated_ds[key].add(entry)
                break
        else:
            continue
        break

In [0]:
tag_idx_dict

In [4]:
import pickle
a_file = open("tag_idx_dict.pkl", "rb")
tag_idx_dict = pickle.load(a_file)
for key, induces in tag_idx_dict.items():
    print(key)

BD
BP
PR
SP
CH
ED


In [0]:
# creation of subsets by choosing proportions of the indices
# concatenation subsets to form the dataset 
# training the model
# getting evaluation results
# saving the model
import pickle
ratio = 1.0
a_file = open("tag_idx_dict.pkl", "rb")
tag_idx_dict = pickle.load(a_file)
ds_list = []
for key, indices in tag_idx_dict.items():
    if len(indces) != 0:
        print(key)
        ds_list.append(data.Subset(train_dataset, indices[:int(len(indices)*ratio)]))

train_dataset_seg = data.ConcatDataset(ds_list)
train_iter = data.DataLoader(dataset=train_dataset_seg,
                              batch_size=128,
                              shuffle=True,
                              num_workers=16,
                              collate_fn=pad)
for epoch in range(1, 1+1):
    train(model, train_iter, optimizer, criterion)

In [0]:
#experiment_code
from google.colab import drive
import time
drive.mount('/content/drive')

In [0]:
#experiment_code
precision, recall, f1 = eval(model, man_test_iter, './eval')

In [0]:
#experiment_code
fname = f'/content/drive/My Drive/imrsv/experiments/data_segregation/weights/w_r_wikiner_{int(100.0*ratio)}.pt'
torch.save(model.state_dict(), f"{fname}")
# torch.save(model.state_dict(), 'weights_overall_P0.79_R0.70_F0.74.pt')
print(f"weights were saved to {fname}")

In [0]:
def train(model, iterator, optimizer, criterion):
    model.train()
    for i, batch in enumerate(iterator):
        words, x, is_heads, tags, y, seqlens = batch
        _y = y # for monitoring
        optimizer.zero_grad()
        logits, y, _ = model(x, y) # logits: (N, T, VOCAB), y: (N, T)

        logits = logits.view(-1, logits.shape[-1]) # (N*T, VOCAB)
        y = y.view(-1)  # (N*T,)

        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        if i==0:
            print("=====sanity check======")
            print("words:", words[0])
            print("x:", x.cpu().numpy()[0][:seqlens[0]])
            print("tokens:", tokenizer.convert_ids_to_tokens(x.cpu().numpy()[0])[:seqlens[0]])
            print("is_heads:", is_heads[0])
            print("y:", _y.cpu().numpy()[0][:seqlens[0]])
            print("tags:", tags[0])
            print("seqlen:", seqlens[0])
            print("=======================")

        if i%10==0: # monitoring
            print(f"step: {i}, loss: {loss.item()}")
            # return


In [11]:
import pickle
a_file = open("tag_idx_dict.pkl", "wb")
pickle.dump(tag_idx_dict, a_file)
a_file.close()

a_file = open("tag_idx_dict.pkl", "rb")
output = pickle.load(a_file)
output == tag_idx_dict

True

In [0]:
#experiment_code
federated_train_loader = sy.FederatedDataLoader(train_dataset.federate((bob, alice)),
                              batch_size=128,
                              shuffle=True)
federated_train_iter = federated_train_loader

In [0]:
#experiment_code
from dataset_results import VOCAB, nn
import remote_bert_wikiner
print(torch.cuda.max_memory_allocated(device=None))
batch_size = 32
lr = 0.0001
n_epochs = 30
finetuning = True
top_rnns = False
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = remote_bert_wikiner.Net(top_rnns=top_rnns, vocab_size=len(VOCAB), device=device, finetuning=finetuning).to(device)
print(torch.cuda.max_memory_allocated(device=None))
criterion = nn.CrossEntropyLoss(ignore_index=0)

In [0]:
from torch import optim
from remote_bert_wikiner import VOCAB_list
def federated_train(model, device, iterator, criterion):
    model.train()
    loc = ''
    for i, batch in enumerate(iterator):
        # words, x, is_heads, tags, y, seqlens = batch
        x, y = batch
        if loc != x.location:
            print(f'data location {x.location}')
            if model.location != None:
                model.get()
            model.send(x.location) # <-- NEW: send the model to the right location
            optimizer = optim.Adam(model.parameters(), lr = lr)
            loc = x.location
        x, y = x.to(device), y.to(device)
        # x, y = x.get(), y.get()
        _y = y # for monitoring
        optimizer.zero_grad()
        logits, y, _ = model(x, y) # logits: (N, T, VOCAB), y: (N, T)
        # print(logits.shape[-1])
        logits = logits.view(-1, len(VOCAB_list)) # (N*T, VOCAB), vocab len = #tags*2<I,N> + 0 + <PAD> 
        y = y.view(-1)  # (N*T,)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        if i%10==0: # monitoring
            print(f"step: {i}, loss: {loss.get().item()}")
        # print(f"step: {i}, loss: {loss.get().item()}")
    model.get() # <-- NEW: get the model back

In [0]:
#experiment_code
# from remote_bert_wikiner import federated_train
for epoch in range(1, 1+1):
    federated_train(model, device, federated_train_iter, criterion)
    print(f"=========eval at epoch={epoch}=========")
print(torch.cuda.max_memory_allocated(device=None))

In [0]:
!nvidia-smi